In [ ]:
from functions_collection_copy import *
import random as random

def new_assembly_Prim_x(seqs,score_matrix,gap_cost,starting_node='0', check_integrity=False):
    # Make a matrix to hold pairwise alignment costs for all alignment combinations!
    matrix = np.full((len(seqs), len(seqs)), np.nan)
    # Loop over all pairs
    for i, seq1 in enumerate(seqs):
        for j, seq2 in enumerate(seqs):
            matrix[i, j] = get_cost_2(linear_C(gap_cost, score_matrix, seq1, seq2))
    matrix_for_MST=matrix #copy the matrix, so that we ca keep the old matrix and make a changed version to the "pseudomatrix" version
    matrix_for_MST=convert_to_desired_format_nr_version(matrix_for_MST) #making the "pseudomatrix"
    if starting_node=='mid':
        starting_node=str(np.argmin(matrix.sum(axis=1)))
    elif starting_node=='ex':
        starting_node=str(np.argmax(matrix.sum(axis=1)))
    elif starting_node=='rd':
        starting_node=str(np.random.randint(9,matrix.shape[0]))
    else:
        starting_node=='0'
    min_span_edges_res=find_min_span_edges_Prim(matrix_for_MST, starting_node)
    in_which_MSA_is_it={}
    names=set()
    for element in min_span_edges_res:
        print(element[2], element[3])
        names.add(element[2])
        names.add(element[3])
    in_which_MSA_is_it ={name: [int(name),0] for name in names}
    MSA_list=[[[char] for char in seq] for seq in seqs]
    who_aligned_to_who=[]
    for row in min_span_edges_res:
        node1=row[2]
        node2=row[3]
        who_aligned_to_who.append([node1,node2])
    for element in min_span_edges_res:
            node1=element[2]
            node2=element[3]
            cost=linear_C(gap_cost,score_matrix,seqs[int(node1)],seqs[int(node2)])
            alignment1_str,alignment2_str=linear_backtrack(seqs[int(node1)], seqs[int(node2)], cost, score_matrix, gap_cost)
            alignment1, alignment2 = [*alignment1_str], [*alignment2_str]
            A = [list(e) for e in zip(alignment1,alignment2)]
            united_MSA_new=alt_alt_merge_united(A,MSA_list,in_which_MSA_is_it,node1,node2)
            in_which_MSA_is_it, MSA_list=perform_updates_OBO(in_which_MSA_is_it, node1, node2, united_MSA_new, MSA_list)
    total_cost = compute_cost(MSA_list[0], score_matrix, gap_cost)
    if check_integrity==True:
       integrity_check_OBO_and_gradual(seqs,in_which_MSA_is_it,who_aligned_to_who,MSA_list, matrix,score_matrix,gap_cost)
    total_cost = compute_cost(MSA_list[0], score_matrix, gap_cost)
    return(matrix,MSA_list, total_cost)

